<a href="https://colab.research.google.com/github/COTILab/MCX24Workshop/blob/master/Training/MCX2024_1E_pmcx_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Workshop Logo](https://mcx.space/wiki/upload/mcx24_logo.png)
# MCX Training Workshop 2024 - Day 1, Session 1E


## Session 1E: PMCX Training

> **Training Goals**: <font color='green'>In this unit, we show you how to use MCX's Python binding, `pmcx`, to create simulations in the Python environments.Particularly, we show visualization of time-resolved data, partial pathlength calculations and photon replay.</font>

# <font color='orange'>Step 0: Setting up environments within Google Colab</font>

> **You must rerun all cells in Step 0 in appearance order when you start a new session, or after reconnect to a runtime**

<font color='green'>If you run this on Google Colab, please go to menu **Edit\Notebook Settings\Hardware accelerator**, and verify if it has already selected "GPU"; if not, please select the T4 GPU.</font>

**Please note**: MCX/MCXLAB is GPU-accelerated. This notebook by default requests an NVIDIA GPU. Howevever, when you click on the run-button for the below section, your browser may fail to allocate a GPU runtime and ask you if you want to continue with a non-GPU runtime. If you choose to proceed without GPU support, you will have to run the [section immediately following the next section](#mcx_setup_opencl) to install OpenCL-based MCXLAB (called mcxlabcl) and utilize the CPU to run the rest of the tutorials. The OpenCL version of MCXLAB will work for all examples, but will be substentially slower to run (this tutorial also reduces the photon numbers accordingly to lower the runtime).

### <font color="orange"> Init 1: Python packages installation</font>

In [ ]:
!pip install pmcx jdata
!git clone https://github.com/fanyuyen/MCXWorkshop2024pyPlot.git

### <font color = "orange"> Init 2: Import Python packages

In [ ]:
import pmcx            ## load pmcx module, the actual binary module is `_pmcx`, which exports two core functions - gpuinfo() and run()
import numpy as np
import jdata as jd
from matplotlib import pyplot as plt
from MCXWorkshop2024pyPlot import plot_3d_slices
import copy

pmcx.__version__   # print imported pmcx version number
#dir(pmcx)         # listing all functions in pmcx module

In [ ]:
# show GPU information
pmcx.gpuinfo()

# Introduction to Python and `pmcx`
`pmcx` is a Python bindings for Monte Carlo eXtreme photon transport simulator.

* Pros of using Python:
  * It is free, unlike MATLAB
  * It offers similar data structures; for instance, MATLAB's arrays can be compared to numpy arrays in Python, and MATLAB's structs are similar to dictionaries in Python.
  * New neuroscience analysis tools are being developed in Python, such as [MNE-Python](https://https://mne.tools/stable/index.html)
  * Machine learning tools are mostly in Python (e.g. [Pytorch](https://pytorch.org/), [Tensorflow](https://www.tensorflow.org/)). Check out one of our previous paper about using deep learning on Monte Carlo simulations [here](https://doi.org/10.1117/1.jbo.27.8.083019).

* Cons of using Python (difference between Python and MATLAB):
  * MATLAB uses column-major (F-order) order for arrays, while Python uses row-major (C-order) order - `pmcx` automatically converts C-order arrays to F-order arrays which is the array order used internally in MCX.
  * MATLAB array index starts from 1, whereas Python/numpy array index starts from 0.
  * In MCXLAB, the number of requested output parameters automatically decides whether to save detected photon, or seeds or trajectories; unfortunately Python does not expose output variable number to a function; therefore, to enable/disable certain outputs, one must sets the appropriate cfg flags (such as `cfg['issavedet']` or `cfg['issaveseed']`.

`pmcx` allows users to run mcx simulation in Python environment, seamlessly connecting with the data analysis tools provided in Python. `pmcx` also handles both column-major input and row-major input.

There are two main ways to run `pmcx`:
1. using cfg as a dictionary variable:
```python
import pmcx
cfg = {
       'nphoton': 1000000,
       'vol':np.ones([60,60,60],dtype='uint8'),
       'tstart':0,
       'tend':5e-9,
       'tstep':5e-9,
       'srcpos': [30,30,0],
       'srcdir':[0,0,1],
       'prop':[[0,0,1,1],[0.005,0.1,0.01,1.37]]
       }
res = pmcx.mcxlab(cfg)
```
2. Positional parameters:
```python
import pmcx
res = pmcx.run(nphoton=1000000, vol=np.ones([60, 60, 60], dtype='uint8'),
               tstart=0, tend=5e-9, tstep=5e-9, srcpos=[30,30,0], srcdir=[0,0,1],
               prop=np.array([[0, 0, 1, 1], [0.005, 1, 0.01, 1.37]]))
```


In this section, we will cover:
* Time-resolved photon transport simulation
* Partial pathlength calculation
* Using replay to generate Jacobian matrix
* Data transfer between Octave/MATLAB and Python

------

# Exec 1 - Time-resolved photon transport simulation example

In this exercise, we will use `pmcx` to demonstrate an example of time-resolved photon transport simulation. This exercise shows how light propagates through a medium and how it interacts with the medium over time.


In this example, you will learn:
* <b>Effects of different time gates: </b> </br>
  Understand how varying the time gates affects the analysis of photon propagation through the medium
* <b>Visualization of light propagation: </b> </br>
  Visualized how light propagates through a homogenous medium over time
* <b>Temporal point spread function (TPSF) curve: </b> </br>
  Plot the temporal point spread function (TPSF) curve at an assigned point in the medium to demonstrate the temporal resolution of the system. This curve shows the distribution of photon arrival times.

In [ ]:
# Photon numbers
nphoton = 1e7
# Simulation domain volumne
vol = [60,60,60]
# starting time, time-gate width, and ending time of the simulation
tstart = 0
tend = 5e-9
# @markdown Number of time gates
ngate = 50 # @param [10, 20, 50, 100] {type:"raw"}
tstep =  tend/ngate
# Source position
srcpos = [30,30,0]
# Source direction
srcdir = [0,0,1]
# Optical properties [$\mu_a$, $\mu_s$, g, n]
prop = [[0,0,1,1],[0.005,0.5,0.01,1.37]]

cfg = {
       'nphoton': nphoton,
       'vol':np.ones(vol,dtype='uint8'),
       'tstart':tstart,
       'tend':tend,
       'tstep':tstep,
       'srcpos': srcpos,
       'srcdir': srcdir,
       'prop':prop
       }

In [ ]:
# Run simulation
res = pmcx.mcxlab(cfg)

In [ ]:
# @title Plot simulation result
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

axs[0].imshow(np.log10(np.sum(res['flux'], axis=3)[29,:, :]))
axs[0].set_title('Sum across all time steps')

axs[1].imshow(np.log10(res['flux'][29,:, :, 0]))
axs[1].set_title('First time step')

axs[2].imshow(np.log10(res['flux'][29,:, :, 1]))
axs[2].set_title('Second time step')

plt.tight_layout()
plt.show()

In [ ]:
# @title Animate the simulation result
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

data = res['flux'][29, :, :, :]

# Calculate the overall min and max values for the color range
with np.errstate(divide="ignore"):
    log_data = np.log10(data)
vmin = np.min(log_data[np.isfinite(log_data)])
vmax = np.max(log_data[np.isfinite(log_data)])

# Set up the figure and axis
fig, ax = plt.subplots()
cax = ax.matshow(log_data[:, :, 0], cmap='viridis', vmin=vmin, vmax=vmax)
fig.colorbar(cax)

def update(frame):
    ax.clear()
    cax = ax.matshow(log_data[:, :, frame], cmap='viridis', vmin=vmin, vmax=vmax)
    ax.set_title(f'Time Step {frame}')
    return cax,

anim = FuncAnimation(fig, update, frames=range(log_data.shape[2]), blit=False)

# Display the animation in the notebook
HTML(anim.to_jshtml())

In [ ]:
# @title Plot result in 3d
# @markdown Choose a timestep to visualize in 3D
time_step = 2 # @param

plot_3d_slices(np.log10(res['flux'][:, :, :, time_step - 1]))

## Plot temporal point spread function (TPSF)



In [ ]:
#@markdown Plot the TPSF of a point at following location (range 0~59):
x = 30 # @param {type:"integer"}
y = 30 # @param {type:"integer"}
z = 30 # @param {type:"integer"}
plt.plot(res['flux'][x,y,z,:], '-o')
plt.title(f"TPSF at [{x},{y},{z}]")
plt.show()

# Exec 2 - Partial pathlength calculation

This example will demonstrate how to calculate partial pathlength in a heterogenous medium with multiple optical properties. Calculating the partial pathlength in each medium allows us to determine the distance that photons travel within specific regions before they are detected. This help us understand how photons interact with different sections of the medium.


In this section, we will cover:
* <b>Use `pmcx` to do multi-medium photon simulation </b>
* <b>Calculate mean partial pathlength in each media: </b></br>
  Understand how to compute the average distance photons travel within each region of the heterogenous medium.
* <b>Plot the histogram of the partial pathlengths: </b></br>
  Visualize the distribution of partial path lengths for each medium to analyze the variability of photon interactions in different regions

In [ ]:
nphoton = 1e7
vol = [60,60,60]
tstart = 0
tend = 5e-9
tstep = 5e-9
srcpos = [30,30,0]
srcdir = [0,0,1]
prop = [[0,0,1,1], [0.005, 1, 0, 1.37], [0.2, 10, 0.9, 1.37], [0.08, 40, 0.9, 1.37]]
detpos = [[30, 20, 1, 1], [30, 40, 1, 1], [20, 30, 1, 1], [40, 30, 1, 1]]

# @markdown Add inclusion - medium 2 (box inclusion)
box_center_x = 30 # @param {type:"integer"}
box_center_y = 30 # @param {type:"integer"}
box_center_z = 20 # @param {type:"integer"}
box_length_x = 20 # @param {type:"integer"}
box_length_y = 20 # @param {type:"integer"}
box_length_z = 20 # @param {type:"integer"}

# @markdown Add inclusion - medium 3 (layer)
layer_axis = 'z' # @param ['x', 'y', 'z']
layer_start = 5 # @param {type:"integer"}
layer_end = 7 # @param {type:"integer"}


cfg = {
       'nphoton': nphoton,
       'vol':np.ones(vol,dtype='uint8'),
       'tstart':tstart,
       'tend':tend,
       'tstep':tend,
       'srcpos': srcpos,
       'srcdir': srcdir,
       'prop':prop,
       'detpos': detpos
       }
if layer_axis == 'x':
    cfg['vol'][layer_start: layer_end, :, :] = 3
if layer_axis == 'y':
    cfg['vol'][:, layer_start: layer_end, :] = 3
if layer_axis == 'z':
    cfg['vol'][:, :, layer_start: layer_end] = 3
cfg['vol'][box_center_x-int(box_length_x/2):box_center_x+int(box_length_x/2), box_center_y-int(box_length_y/2):box_center_y+int(box_length_y/2), box_center_z-int(box_length_z/2):box_center_z+int(box_length_z/2)] = 2

In [ ]:
# Run simulation
res = pmcx.mcxlab(cfg)

In [ ]:
plot_3d_slices(np.log10(res['flux']))

## Calculate mean partial pathlength in each medium

The mean partial path length (MPPL) for the $j$-th region is calculated using the following formula:

$\text{MPPL}_j = \frac{\sum_{i=1}^{N} w_i \cdot L_{i,j}}{\sum_{i=1}^{N} w_i}$

where:
* $N$ is the total number of photon detected
* $w_i$ is the weight associated with the $i$-th photon (can obtain from `pmcx.detweight` function)
* $L_{i,j}$ is the patial path length of the $i$-th photon within the $j$-th region (`res['detp']['ppath']`)

In [ ]:
mppl = np.sum(res['detp']['ppath'] * pmcx.detweight(res['detp'])[:, None], axis=0) / np.sum(pmcx.detweight(res['detp']))
print("Mean partial pathlength in medium 1 (larger cube):, ", mppl[0])
print("Mean partial pathlength in medium 2 (box inclusion):, ", mppl[1])
print("Mean partial pathlength in medium 3 (layer):, ", mppl[2])

## Plot partial pathlength

In [ ]:
n_photons = res['detp']['ppath'].shape[0]

plt.hist(res['detp']['ppath'][:,0], bins=100, alpha=0.5, color = 'blue', label='Medium1 (larger cube)')
plt.hist(res['detp']['ppath'][:,1], bins=100, alpha=0.5, color = 'green', label='Medium2 (box inclusion)')
plt.hist(res['detp']['ppath'][:,2], bins=100, alpha=0.5, color = 'red', label='Medium3 (layer)')
plt.xlabel("partial pathlength")
plt.title(f"detected {n_photons} photons")
plt.legend()
plt.show()

# Exec 3 - Replay in photon simulation
In this example, we will show the replay in photon simulations. Replay involves storing photon paths and previously recorded seeds. By saving the seeds, replay can create Jacobians matrix for absorption and scattering coefficients, which is crucial for applications like diffuse optical tomography (DOT).

![Photon replay](https://opg.optica.org/getimagev2.cfm?img=gn%2B3%2Fk27EUBH1o4wZKu2dSIRvRy2Zv%2FXywcVAYmGSyU%3D&size=full&uri=boe-9-10-4588-g001)

We will cover the following topics:
* <b>Use replay to generate the Jaobian matrix: </b></br>
  Learn how to use replay in photon simulation to generate a Jacobian matrix.

* <b>Data exchange between Octave/MATLAB and Python: </b></br>
  Understand the process of exchanging data files between Octave/MATLAB and Python. This allows users to perform simulation in the Octave/MATLAB environment and then visualize or analyze the results in the Python environment, using Python's libraries for machine learning and data analysis.



Reference: [Replay paper](https://doi.org/10.1364/BOE.9.004588)

In [ ]:
# @title Setting up octave environment for running mcxlab
!apt-get install octave      # install octave to Linux host
!pip install oct2py          # install oct2py Python module

# add octave support to colab notebook
%load_ext oct2py.ipython

In [ ]:
# download and unzip mcxlab
!rm -rf mcxlab*
!wget http://mcx.space/nightly/release/v2023/mcxlab-allinone-v2023.zip  # download mcxlab
!unzip mcxlab-allinone-v2023.zip && rm -rf mcxlab-allinone-v2023.zip # unzip mcxlab

# download jsonlab and zmat toolboxes for sharing data between MATLAB and Python
!git clone https://github.com/fangq/jsonlab.git  # download jsonlab
!git clone https://github.com/NeuroJSON/zmat.git # download zmat

In [ ]:
%%octave
addpath([pwd filesep 'mcxlab']);                 % add path to mcx
addpath([pwd filesep 'mcxlab' filesep 'utils']); % add path to mcx helper functions
addpath([pwd filesep 'jsonlab']);                % optional: add path to jsonlab for data export
addpath([pwd filesep 'zmat']);                   % optional: add path to zmat for data compression

In [ ]:
# @title Initial simulation (Baseline MC)
%%octave
clear cfg

cfg.nphoton=1e7;
cfg.vol=uint8(ones(60,60,60));
% cfg.vol(20:40,20:40,10:30)=2;
cfg.tstart=0;
cfg.tend=5e-9;
cfg.tstep=5e-9;
cfg.srcpos=[30 30 1];
cfg.srcdir=[0 0 1];
cfg.prop=[0 0 1 1;
          0.005 1 0.01 1.37;
          0.1, 10, 0.9, 1];
cfg.detpos=[30,20,0,1];            % to detect photons, one must first define detectors
cfg.issavedet=1;                   % cfg.issavedet must be set to 1 or True in order to save detected photons
cfg.issrcfrom0=1;                  % set this flag to ensure src/det coordinates align with voxel space
cfg.issaveseed=1;     %%!important!% set this flag to store detected photon seed data

[fluence,detphoton,vol,seeds]=mcxlab(cfg);

In [ ]:
# @title Replay MC
%%octave
cfg_replay = cfg;
cfg_replay.seed = seeds.data;
cfg_replay.detphotons = detphoton;
cfg_replay.outputtype = 'jacobian';

[fluence2,detphoton2,vol2,seed2] = mcxlab(cfg_replay);

In [ ]:
%%octave
% save to text-based JSON file and list file size
tic;savejd('', fluence2, 'filename', 'mcx_replay_jacobian.json', 'compression', 'zlib');toc
system('ls -l mcx_replay_jacobian.json');

## Load simulation result in python

In [ ]:
flux=jd.load('mcx_replay_jacobian.json')

plt.imshow(np.log10(flux['data'][29,:,:])) # Not 30, cause Python index start from 0
plt.title("Replay jacobian")
plt.show()

In [ ]:
plot_3d_slices(np.log10(flux['data']))

---

Note: `pmcx` is still being developed. If you're missing any features from MATLAB that aren't in Python yet, or if you find any bugs while using `pmcx` toolbox, please let us know!

The [MCX utility MATLAB functions](https://github.com/fangq/mcx/tree/master/utils) that have been ported to pmcx include the following

In [25]:
dir(pmcx)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'bench',
 'cwdref',
 'detphoton',
 'dettime',
 'dettpsf',
 'detweight',
 'getdistance',
 'gpuinfo',
 'mcxlab',
 'meanpath',
 'meanscat',
 'run',
 'tddiffusion',
 'utils',
 'version']